# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6h Sleeping whole brain  C57BL/6J (B6) strain Expression values and Group 2 be 6hSD whole brain  C57BL/6J (B6) strain Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_WB_B6_'   # define a prefix to add to column names (mB6ing indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9441.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 51)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 51)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239832_AK_S.CEL', 'GSM239833_AK_S.CEL',
       'GSM239834_AK_S.CEL', 'avg-AK_S', 'log_fold-AK_S_vs_AK_SD',
       'fold-AK_S_vs_AK_SD', 'rawp-AK_S_vs_AK_SD', 'adjp-AK_S_vs_AK_SD',
       'GSM239835_AK_SD.CEL', 'GSM239836_AK_SD.CEL', 'GSM239837_AK_SD.CEL',
       'avg-AK_SD', 'GSM239838_B6_S.CEL', 'GSM239839_B6_S.CEL',
       'GSM239840_B6_S.CEL', 'avg-B6_S', 'log_fold-B6_S_vs_B6_SD',
       'fold-B6_S_vs_B6_SD', 'rawp-B6_S_vs_B6_SD', 'adjp-B6_S_vs_B6_SD',
       'GSM239841_B6_SD.CEL', 'GSM239842_B6_SD.CEL', 'GSM239843_B6_SD.CEL',
       'avg-B6_SD', 'GSM239844_D2_S.CEL', 'GSM239845_D2_S.CEL',
       'GSM239846_D2_S.CEL', 'avg-D2_S', 'log_fold-D2_S_vs_D2_SD',
       'fold-D2_S_vs_D2_SD', 'rawp-D2_S_vs_D2_SD', 'adjp-D2_S_vs_D2_SD',
       'G

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='B6_S.CEL'
sd_filt ='B6_SD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239838_B6_S.CEL,GSM239839_B6_S.CEL,GSM239840_B6_S.CEL
Probesets,,,
1427138_at,5.00722,5.22060,4.78427
1425600_a_at,6.81027,5.47446,6.82511
1457168_at,4.23937,4.15903,4.22013
1450135_at,4.06382,4.03176,4.11579
1424014_at,7.64870,7.51508,7.56990


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239841_B6_SD.CEL,GSM239842_B6_SD.CEL,GSM239843_B6_SD.CEL
Probesets,,,
1427138_at,4.89095,4.93817,4.90112
1425600_a_at,6.67312,6.52716,6.74435
1457168_at,4.70188,4.08710,4.12907
1450135_at,4.63504,4.02510,4.34277
1424014_at,7.33480,7.64282,7.77544


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.093950
1425600_a_at    0.278263
1457168_at      0.099840
1450135_at      0.263847
1424014_at      0.006460
Name: 430AV2_WB_B6_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.605052
1425600_a_at    0.502320
1457168_at      0.408053
1450135_at      1.211512
1424014_at      0.038739
Name: 430AV2_WB_B6_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.514294
1425600_a_at    0.426972
1457168_at      0.346845
1450135_at      1.029786
1424014_at      0.032928
Name: 430AV2_WB_B6_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.697174
1425600_a_at    0.687694
1457168_at      0.680542
1450135_at      0.788980
1424014_at      0.666792
Name: 430AV2_WB_B6_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_WB_B6_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_WB_B6_Enrich,430AV2_WB_B6_poolStDev,430AV2_WB_B6_Cohens_d,430AV2_WB_B6_J,430AV2_WB_B6_Hedges_g,430AV2_WB_B6_Var_d,430AV2_WB_B6_Var_g,430AV2_WB_B6_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1416064_a_at,Hspa5,heat shock protein 5,ENSMUSG00000026864,14828,NaN,response to stress // ER overload response,protein binding // nucleotide binding // ATP b...,endoplasmic reticulum lumen // melanosome // e...,MAPK signaling pathway:WP493(WikiPathways),"mmu-miR-1192(TargetScan|miRanda), mmu-miR-1193...",...,1.191694e-07,1.365547,0.812097,0.032910,24.676346,0.85,20.974894,51.410172,37.143849,6.094575
1460179_at,Dnaja1,"DnaJ (Hsp40) homolog, subfamily A, member 1",ENSMUSG00000028410,15502,NaN,androgen receptor signaling pathway // spermat...,heat shock protein binding // metal ion bindin...,membrane,Diurnally Regulated Genes with Circadian Ortho...,"mmu-miR-103(miRanda), mmu-miR-107(miRanda), mm...",...,6.948974e-04,0.302483,0.141120,0.013263,10.640440,0.85,9.044374,10.101581,7.298392,2.701554
1417606_a_at,Calr,calreticulin,ENSMUSG00000003814|ENSMUSG00000081731,12317,NaN,cortical actin cytoskeleton organization // se...,calcium ion binding // sugar binding // zinc i...,extracellular space // MHC class I peptide loa...,Calcium Regulation in the Cardiac Cell:WP553(W...,"mmu-let-7b(RNAhybrid|miRanda), mmu-let-7c(RNAh...",...,1.205508e-05,0.765267,0.323587,0.030437,10.631397,0.85,9.036687,10.085549,7.286809,2.699409
1447433_at,Wdfy3,WD repeat and FYVE domain containing 3,ENSMUSG00000043940,72145,Mm.332522,NaN,"beta-N-acetylglucosaminylglycopeptide beta-1,4...",nuclear envelope // autophagic vacuole // cyto...,NaN,"mmu-let-7a(RNAhybrid|pictar), mmu-let-7b(RNAhy...",...,1.343724e-01,0.201033,0.201033,0.019120,10.514474,0.85,8.937303,9.879514,7.137949,2.671694
1429564_at,LOC100048247 /// Pcgf5,polycomb group ring finger 5 [Source:MGI Symbo...,ENSMUSG00000024805,76073|100048247,NaN,"transcription // regulation of transcription, ...",protein binding // metal ion binding // zinc i...,centrosome,NaN,"mmu-miR-101(TargetScan), mmu-miR-101a(TargetSc...",...,6.657891e-02,0.652397,0.211283,0.021139,9.994906,0.85,8.495670,8.991512,6.496368,2.548797
1424638_at,Cdkn1a,cyclin-dependent kinase inhibitor 1A (P21),ENSMUSG00000023067,12575,NaN,cellular response to extracellular stimulus //...,cyclin binding // zinc ion binding // kinase a...,cyclin-dependent protein kinase holoenzyme com...,GenMAPP-Cell_Cycle_KEGG // PluriNetWork:WP1763...,"mmu-miR-105(TargetScan), mmu-miR-106a(TargetSc...",...,1.578591e-03,1.054710,0.855187,0.085771,9.970547,0.85,8.474965,8.950983,6.467086,2.543047
1426908_at,Galnt7,UDP-N-acetyl-alpha-D-galactosamine: polypeptid...,ENSMUSG00000031608,108150,NaN,protein amino acid O-linked glycosylation,"calcium ion binding // transferase activity, t...",Golgi membrane // Golgi apparatus // integral ...,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,1.748065e-02,0.634383,0.338713,0.034098,9.933583,0.85,8.443546,8.889673,6.422788,2.534322
1447402_at,NaN,NaN,NaN,NaN,Mm.397136,NaN,NaN,NaN,NaN,NaN,...,3.294001e-01,0.254320,0.254320,0.025614,9.928786,0.85,8.439468,8.881733,6.417052,2.533190
1424147_at,Ahsa1,"AHA1, activator of heat shock protein ATPase 1",ENSMUSG00000021037,217737,NaN,response to stress // protein folding,protein binding // ATPase activator activity /...,cytoplasm // endoplasmic reticulum,NaN,"mmu-miR-100(RNAhybrid), mmu-miR-125a(RNAhybrid...",...,1.119300e-03,0.530113,0.240043,0.025224,9.516414,0.85,8.088952,8.213512,5.934262,2.436034


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239832_AK_S.CEL', 'GSM239833_AK_S.CEL',
       'GSM239834_AK_S.CEL', 'avg-AK_S', 'log_fold-AK_S_vs_AK_SD',
       'fold-AK_S_vs_AK_SD', 'rawp-AK_S_vs_AK_SD', 'adjp-AK_S_vs_AK_SD',
       'GSM239835_AK_SD.CEL', 'GSM239836_AK_SD.CEL', 'GSM239837_AK_SD.CEL',
       'avg-AK_SD', 'GSM239838_B6_S.CEL', 'GSM239839_B6_S.CEL',
       'GSM239840_B6_S.CEL', 'avg-B6_S', 'log_fold-B6_S_vs_B6_SD',
       'fold-B6_S_vs_B6_SD', 'rawp-B6_S_vs_B6_SD', 'adjp-B6_S_vs_B6_SD',
       'GSM239841_B6_SD.CEL', 'GSM239842_B6_SD.CEL', 'GSM239843_B6_SD.CEL',
       'avg-B6_SD', 'GSM239844_D2_S.CEL', 'GSM239845_D2_S.CEL',
       'GSM239846_D2_S.CEL', 'avg-D2_S', 'log_fold-D2_S_vs_D2_SD',
       'fold-D2_S_vs_D2_SD', 'rawp-D2_S_vs_D2_SD', 'adjp-D2_S_vs_D2_SD',
       'G

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239832_AK_S.CEL,GSM239833_AK_S.CEL,GSM239834_AK_S.CEL,avg-AK_S,log_fold-AK_S_vs_AK_SD,fold-AK_S_vs_AK_SD,rawp-AK_S_vs_AK_SD,adjp-AK_S_vs_AK_SD,GSM239835_AK_SD.CEL,GSM239836_AK_SD.CEL,...,ANOVA-adjp,largest fold,430AV2_WB_B6_Enrich,430AV2_WB_B6_poolStDev,430AV2_WB_B6_Cohens_d,430AV2_WB_B6_J,430AV2_WB_B6_Hedges_g,430AV2_WB_B6_Var_d,430AV2_WB_B6_Var_g,430AV2_WB_B6_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_WB_B6_Enrich',u'430AV2_WB_B6_Hedges_g', u'430AV2_WB_B6_Var_g', u'430AV2_WB_B6_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_WB_B6_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_WB_B6_Enrich,430AV2_WB_B6_Hedges_g,430AV2_WB_B6_Var_g,430AV2_WB_B6_SEg
MGI symbol,,,,
0610005C13Rik,0.020797,0.168026,0.484019,0.695715
0610008F07Rik,0.161643,0.428478,0.496966,0.704958
0610009B22Rik,-0.014903,-0.076177,0.482150,0.694370
0610009D07Rik,-0.120952,-0.697805,0.522553,0.722862
0610009O20Rik,-0.038557,-0.276604,0.488042,0.698600
0610010K14Rik,-0.294430,-0.763285,0.530217,0.728160
0610012G03Rik,-0.148063,-0.523543,0.511791,0.715152
0610031J06Rik,-0.047600,-0.915513,0.551514,0.742640
0610037L13Rik,0.036733,0.291984,0.488771,0.699122
